In [3]:
from greenseer.configuration import do_global_configuration,get_global_configuration
do_global_configuration("config.properties")

In [1]:
from greenseer.repository.china_stock import  create_china_stock_assert_repository
from logging.config import fileConfig
fileConfig('logging_config.ini')
china_daily_repository =  create_china_stock_assert_repository()
data = china_daily_repository.load_data("600096").T
data.shape

(52, 108)